In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv("C:/Users/User/Desktop/kshi/crop/karcrop.csv",sep=",")
data = data.fillna(data.mean())
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21122 entries, 0 to 21121
Data columns (total 7 columns):
State_Name       21122 non-null object
District_Name    21122 non-null object
Crop_Year        21122 non-null int64
Season           21122 non-null object
Crop             21122 non-null object
Area             21122 non-null int64
Production       21122 non-null float64
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ MB


In [3]:
print(data.head())

  State_Name District_Name  Crop_Year       Season          Crop   Area  \
0  Karnataka      BAGALKOT       1998  Kharif          Arhar/Tur   6154   
1  Karnataka      BAGALKOT       1998  Kharif              Bajra  48855   
2  Karnataka      BAGALKOT       1998  Kharif        Castor seed     71   
3  Karnataka      BAGALKOT       1998  Kharif       Cotton(lint)  15225   
4  Karnataka      BAGALKOT       1998  Kharif          Groundnut  16368   

   Production  
0      2602.0  
1     52375.0  
2        61.0  
3     22129.0  
4      7734.0  


In [4]:
print(data.describe())

          Crop_Year           Area    Production
count  21122.000000   21122.000000  2.112200e+04
mean    2006.199508    9606.577171  4.096161e+04
std        4.989631   26295.617812  4.265591e+05
min     1997.000000       1.000000  0.000000e+00
25%     2002.000000     100.000000  1.160000e+02
50%     2006.000000     839.500000  9.340000e+02
75%     2011.000000    5419.750000  7.473750e+03
max     2014.000000  510692.000000  1.802747e+07


In [5]:
def plot_graphs(groundtruth,prediction,title):        
    N = 9
    ind = np.arange(N)  # the x locations for the groups
    width = 0.27       # the width of the bars

    fig = plt.figure()
    fig.suptitle(title, fontsize=12)
    ax = fig.add_subplot(111)
    rects1 = ax.bar(ind, groundtruth, width, color='r')
    rects2 = ax.bar(ind+width, prediction, width, color='g')

    ax.set_ylabel("Amount of rainfall")
    ax.set_xticks(ind+width)
    ax.set_xticklabels( ('APR', 'MAY', 'JUN', 'JUL','AUG', 'SEP', 'OCT', 'NOV', 'DEC') )
    ax.legend( (rects1[0], rects2[0]), ('Ground truth', 'Prediction') )

#     autolabel(rects1)
    for rect in rects1:
        h = rect.get_height()
        ax.text(rect.get_x()+rect.get_width()/2., 1.05*h, '%d'%int(h),
                ha='center', va='bottom')
    for rect in rects2:
        h = rect.get_height()
        ax.text(rect.get_x()+rect.get_width()/2., 1.05*h, '%d'%int(h),
                ha='center', va='bottom')
#     autolabel(rects2)

    plt.show()

In [6]:
# seperation of training and testing data
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

division_data = np.asarray(data[['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', 'NOV', 'DEC']])

X = None; y = None
for i in range(division_data.shape[1]-3):
    if X is None:
        X = division_data[:, i:i+3]
        y = division_data[:, i+3]
    else:
        X = np.concatenate((X, division_data[:, i:i+3]), axis=0)
        y = np.concatenate((y, division_data[:, i+3]), axis=0)
        
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.03, random_state=42)

In [7]:
temp = data[['DISTRICT','JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL','AUG', 'SEP', 'OCT', 'NOV', 'DEC']].loc[data['STATE_UT_NAME'] == 'KARNATAKA']
kar = np.asarray(temp[['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL','AUG', 'SEP', 'OCT', 'NOV', 'DEC']].loc[temp['DISTRICT'] == 'KARNATAKA'])
# print temp
X_year = None; y_year = None
for i in range(kar.shape[1]-3):
    if X_year is None:
        X_year = kar[:, i:i+3]
        y_year = kar[:, i+3]
    else:
        X_year = np.concatenate((X_year, kar[:, i:i+3]), axis=0)
        y_year = np.concatenate((y_year, kar[:, i+3]), axis=0)

In [8]:
from sklearn.svm import SVR

# SVM model
clf = SVR(gamma='auto', C=0.1, epsilon=0.2)
clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)
print(mean_absolute_error(y_test, y_pred))
print(y_pred)

53.846026515406045
[80.1359364  80.15166311 80.15166321 80.15168171 80.15166311 80.15166311
 80.15166311 80.10518307 80.15166311]


In [9]:
y_year_pred = clf.predict(X_year)
print("MEAN KARNATAKA")
print(np.mean(y_year),np.mean(y_year_pred))
print("Standard deviation KARNATAKA")
print(np.sqrt(np.var(y_year)),np.sqrt(np.var(y_year_pred)))

plot_graphs(y_year,y_year_pred,"Prediction in KARNATAKA")

ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required.

In [ ]:
from keras.models import Model
from keras.layers import Dense, Input, Conv1D, Flatten

# NN model
inputs = Input(shape=(3,1))
x = Conv1D(64, 2, padding='same', activation='elu')(inputs)
x = Conv1D(128, 2, padding='same', activation='elu')(x)
x = Flatten()(x)
x = Dense(128, activation='elu')(x)
x = Dense(64, activation='elu')(x)
x = Dense(32, activation='elu')(x)
x = Dense(1, activation='linear')(x)
model = Model(inputs=[inputs], outputs=[x])
model.compile(loss='mean_squared_error', optimizer='adamax', metrics=['mae'])
model.summary()

In [ ]:
model.fit(x=np.expand_dims(X_train, axis=2), y=y_train, batch_size=64, epochs=10, verbose=1, validation_split=0.1, shuffle=True)
y_pred = model.predict(np.expand_dims(X_test, axis=2))
print(mean_absolute_error(y_test, y_pred))

In [ ]:
print(data.groupby("DISTRICT").sum()['ANNUAL'].plot(figsize=(10,4)))

In [ ]:
data[['DISTRICT', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', 'NOV', 'DEC']].groupby("DISTRICT").sum().plot(figsize=(10,5));

In [ ]:
data[['DISTRICT', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL',
       'AUG', 'SEP', 'OCT', 'NOV', 'DEC']].groupby("DISTRICT").mean().plot.barh(stacked=True,figsize=(5,6));

In [ ]:
data[['DISTRICT', 'Jan-Feb', 'Mar-May',
       'Jun-Sep', 'Oct-Dec']].groupby("DISTRICT").sum().plot.barh(stacked=True,figsize=(5,7));

In [ ]:
plt.figure(figsize=(7,4))
sns.heatmap(data[['Jan-Feb','Mar-May','Jun-Sep','Oct-Dec','ANNUAL']].corr(),annot=True)
plt.show()